In [13]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from data.random_data import RandomOHLCV

ohlcv = RandomOHLCV( 
    freq      = '15 min', 
    head_max  = 0.3, 
    tail_max  = 0.3, 
    start     = '2024',           
    open_val  = 100.00,           
    periods   = 300, 
    open_rng  = (-0.4, 0.4), 
    close_rng = (-0.4, 0.4), 
    vol_rng   = (-50, 60),
    volatility_rng  = (0, 0.02),
    volatility_dur  = 3,
    volatility_freq = 50
).get_dataframe()

ohlcv.tail()

,open,high,low,close,volume
date,,,,,
2024-01-04 01:45:00,113.72,114.21,113.58,113.90,500
2024-01-04 02:00:00,113.58,114.10,113.42,113.94,2566
2024-01-04 02:15:00,114.18,114.48,113.94,114.37,41927
2024-01-04 02:30:00,114.20,114.36,113.74,113.74,500
2024-01-04 02:45:00,113.52,113.94,113.39,113.65,500


In [14]:
from strategies.ta import MA, MACD, HPLP, ATR
from frame.frame import Frame
f = Frame('TSLA')
f.load_ohlcv(ohlcv)
f.setup_chart() 

f.add_ta(MA('close', 9),  {'dash': 'solid', 'color': 'yellow', 'width': 2}) # creates new column to the dataframe with the moving average eg MA_C9
f.add_ta(MA('close', 50), {'dash': 'solid', 'color': 'purple', 'width': 4})
f.add_ta(MA('close', 200), {'dash': 'solid', 'color': 'blue', 'width': 6})
f.add_ta(ATR(span=10), {}, chart_type='')
f.add_ta(MACD(fast=12, slow=26, signal=9) , 
         [{'dash': 'solid', 'color': 'purple', 'width': 2}, # MACD
          {'dash': 'solid', 'color': 'pink', 'width': 2},   # signal
          {'color': 'black'}], 'macd', 2) # histogram

f.add_ta(HPLP(hi_col='high', lo_col='low', span=5), 
        [{'color': 'green', 'size': 10}, # high points
          {'color': 'red', 'size': 10}], # low points
          chart_type = 'points')

f.update_ta_data()
f.plot(trading_hours=True, width=2000, height=1200)

In [15]:
# Create trades (IDs are automatically assigned)
import trade_log    
trade1 = trade_log.TradeLog(
    symbol="AAPL",
    barsize="1min",
    av_entry_price=150.25,
    position=100,
    direction="SHORT",
    # entry_time="2024-04-14 03:15:00",
    entry_time=f.data.index[0],
    chart_time=f.data.index[0],
)

trade1

AttributeError: module 'trade_log' has no attribute 'TradeLog'

In [ ]:
import stops

init_stop = stops.StopPrevBar()

# Create a stop loss instance with your configuration
sl = stops.StopLoss(
    # init=stops.StopPrevBar(),
    init=stops.StopGapDiff(3),
    # trail1=stops.StopPriorPiv(5),
    cond1=stops.CondDuration(5),
    trail1=stops.StopGapDiff(3),
    trail2=stops.StopPriorPiv(5),
    # trail2=stops.StopMA(21),
    cond2=stops.CondRRatio(2)
)

# Get the stop price for a trade
stop_price = sl.get_price(ohlcv, trade1)
stop_price

In [ ]:
f.data['close'].iat[-1]

np.float64(99.48)

In [ ]:
trade1.price_now = f.data['close'].iat[-1]

sl.get_price(ohlcv, trade1)

In [ ]:


import pandas as pd

# Initialize an empty dictionary to store stop prices
stop_prices = {}

# Iterate through the data and calculate stop prices
for i in f.data.index:
    trade1.price_now = f.data.loc[i, 'close']
    trade1.chart_time = i
    stop_price = sl.get_price(ohlcv, trade1)
    stop_prices[i] = stop_price
    # print(f"{i} - {trade1.price_now} - {stop_price}")

# Convert the dictionary to a Pandas Series

pd.Series(stop_prices).tail(30)


2024-01-03 19:30:00    100.465
2024-01-03 19:45:00    100.465
2024-01-03 20:00:00    100.465
2024-01-03 20:15:00    100.465
2024-01-03 20:30:00    100.465
2024-01-03 20:45:00    100.465
2024-01-03 21:00:00    100.465
2024-01-03 21:15:00    100.465
2024-01-03 21:30:00    100.465
2024-01-03 21:45:00    100.465
2024-01-03 22:00:00    100.465
2024-01-03 22:15:00    100.465
2024-01-03 22:30:00    100.465
2024-01-03 22:45:00    100.465
2024-01-03 23:00:00    100.465
2024-01-03 23:15:00    100.465
2024-01-03 23:30:00    100.465
2024-01-03 23:45:00    100.465
2024-01-04 00:00:00    100.465
2024-01-04 00:15:00    100.465
2024-01-04 00:30:00    100.465
2024-01-04 00:45:00    100.465
2024-01-04 01:00:00    100.465
2024-01-04 01:15:00    100.465
2024-01-04 01:30:00    100.465
2024-01-04 01:45:00    100.465
2024-01-04 02:00:00    100.465
2024-01-04 02:15:00    100.465
2024-01-04 02:30:00    100.465
2024-01-04 02:45:00    100.465
dtype: float64

In [ ]:
f.chart.add_line(pd.Series(stop_prices), {'dash': 'solid', 'color': 'red', 'width': 2})
# f.plot(trading_hours=True, width=2000, height=1200)
f.chart.show(width=2000, height=1200)

NameError: name 'stop_prices' is not defined

In [ ]:
import entries as ent

def create_long_entry_strategy() -> ent.EntryStrategy:
    conditions = [
        ent.PriceAbovePreviousHigh(),
        ent.LowerHighsPreviousBars(num_bars=2),
        # ent.PriceNotBelowMA(ma_column='MA_cl_50', atr_column='ATR_10', atr_multiplier=0.0)
    ]
    return ent.EntryStrategy(conditions)


entry1 = create_long_entry_strategy()
entry1.should_enter(f.data, print_results=True)

PriceAbovePreviousHigh: False
LowerHighsPreviousBars: False


False

In [ ]:
f.data.tail(10)

,open,high,low,close,volume,MA_cl_9,MA_cl_21,MA_cl_28,ATR_10,MACD_cl_12_26_9_MACD,MACD_cl_12_26_9_Signal,MACD_cl_12_26_9_Histogram,HP_hi_5,LP_lo_5
date,,,,,,,,,,,,,,
2024-01-04 00:30:00,88.84,89.04,88.50,88.68,500,88.520000,87.926190,87.826786,0.378,0.274574,0.104649,0.169925,NaN,NaN
2024-01-04 00:45:00,89.00,89.46,88.96,89.27,15320,88.680000,87.975238,87.898929,0.416,0.325792,0.148878,0.176914,NaN,NaN
2024-01-04 01:00:00,89.35,89.47,89.01,89.09,500,88.757778,88.027143,87.964643,0.423,0.347848,0.188672,0.159176,NaN,NaN
2024-01-04 01:15:00,89.39,89.60,89.25,89.34,500,88.890000,88.096190,88.036071,0.400,0.381107,0.227159,0.153948,NaN,NaN
2024-01-04 01:30:00,89.13,89.39,89.09,89.15,500,88.986667,88.162857,88.096429,0.408,0.387665,0.259260,0.128405,NaN,NaN
2024-01-04 01:45:00,89.48,89.89,89.28,89.68,500,89.134444,88.262381,88.162143,0.441,0.430664,0.293541,0.137123,NaN,NaN
2024-01-04 02:00:00,89.55,89.97,89.34,89.83,24136,89.244444,88.382857,88.236071,0.452,0.471411,0.329115,0.142296,89.97,NaN
2024-01-04 02:15:00,89.82,89.97,89.69,89.81,500,89.307778,88.510952,88.307857,0.431,0.496368,0.362565,0.133802,NaN,NaN
2024-01-04 02:30:00,89.46,89.63,89.11,89.24,20731,89.343333,88.628571,88.343571,0.473,0.464794,0.383011,0.081783,NaN,NaN


In [ ]:
trader1.trade_details.chart_time

datetime.datetime(2024, 11, 10, 16, 8, 5, 911634)

In [ ]:

from mock_ib import MockIB
import trades, entries, stops

# util.startLoop()  # uncomment this line when in a notebook
# ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=1)

ib = MockIB()
ib.connect('127.0.0.1', 7497, clientId=1)


stoploss = stops.StopLoss(
    init=stops.StopPrevBar(),
    # init=stops.StopGapDiff(3),
    # trail1=stops.StopPriorPiv(5),
    cond1=stops.CondDuration(5),
    trail1=stops.StopGapDiff(3),
    trail2=stops.StopPriorPiv(5),
    # trail2=stops.StopMA(21),
    cond2=stops.CondRRatio(2)
)

entrystrategy = entries.EntryStrategy(
    name='Hot Stuff',
    conditions=[
        entries.PriceAbovePreviousHigh(),
        entries.LowerHighsPreviousBars(num_bars=2),
        # entries.PriceNotBelowMA(ma_column='MA_cl_50', atr_column='ATR_10', atr_multiplier=0.0)
    ]
)

trader1 = trades.BaseTrade(ib, 'TSLA', '5min', entrystrategy, stoploss)
print(stoploss.get_price(f.data, trader1.trade_details))

trader1.find_entry(f.data, force_entry=True)
# trader1.get_trade_history()[-1].to_dict()
trader1.trade_details.__dict__

0.0
Entry filled: None @ 100.0


{'symbol': 'TSLA',
 'barsize': '5min',
 'direction': 'LONG',
 'status': 'ENTERED',
 'is_active': True,
 'chart_time': datetime.datetime(2024, 11, 10, 21, 18, 2, 258058),
 'real_time': datetime.datetime(2024, 11, 10, 21, 18, 2, 258058),
 'entry_time': datetime.datetime(2024, 11, 10, 21, 18, 2, 258058),
 'exit_time': None,
 'duration': None,
 'entry_name': None,
 'av_entry_price': 100.0,
 'ib_entry_id': '0',
 'entry_filled': (None, None),
 'exit_name': None,
 'av_exit_price': None,
 'ib_exit_id': None,
 'exit_filled': (0, 0),
 'exit_type': None,
 'position': None,
 'value': 0.0,
 'close_price': np.float64(99.27),
 'stop_price': None,
 'target_price': None,
 'stop_name': None,
 'target_name': None,
 'risk_reward': None,
 'target_risk_reward': 0.0,
 'unrealized_pl': 0.0,
 'realized_pl': 0.0,
 'precision': 2,
 'risk_percentage': 0.01,
 'account_size': 100000.0,
 'commission': 1.0,
 'slippage': 0.0,
 'total_cost': 1.0,
 'stop_order_id': '0',
 'target_order_id': None,
 'trade_number': 1,
 'lo

In [ ]:
s1 = stops.StopPrevBar()
print(s1.calculate(f.data, trader1.trade_details))

s2 = stops.StopGapDiff(3)
print(s2.calculate(f.data, trader1.trade_details))

print(f"price now {trader1.trade_details.price_now}")   

98.49
98.38
price now 100.0


In [ ]:
import trades as tr



price_now = 50
entry_price = 100
stop_price = 99.5
account_size = 10_000
risk = 0.01
margin_rate = 0.02

pos_size   = tr.get_qty_shares(entry_price, stop_price, account_size, risk)
pos_amount = tr.get_total_trade_value(entry_price, pos_size)     
rr_ratio   = tr.get_rr_ratio(entry_price, stop_price, price_now)
pl         = tr.get_pl(entry_price, exit_price=price_now, pos_size=pos_size, direction='LONG')   
margin_allowance         = tr.get_margin_value(pos_amount, margin_rate)
account_size_with_margin = tr.get_total_available_funds_with_margin(account_size, margin_rate)
exceeds_account  = tr.is_position_exceeding_account(entry_price, pos_size, account_size, margin_rate)
max_pos = tr.max_position_size(entry_price, stop_price, account_size, margin_rate, risk)
#! get max 5 of portfoli size . eg dont put whole protfolio in one stock.  Modify max_position_size

print(f"Position size           : {pos_size}")
print(f"RR ratio                : {rr_ratio}")
print(f"Profit/Loss             : {pl}")
print(f"Position amount         : {pos_amount}")
print(f"Margin allowance        : {margin_allowance}")
print(f"Exceeds account         : {exceeds_account}")
print(f"Account size with margin: {account_size_with_margin}")
print(f"max_pos                 : {max_pos}")

Position size           : 200
RR ratio                : -100.0
Profit/Loss             : -10000
Position amount         : 20000
Margin allowance        : 400.0
Exceeds account         : True
Account size with margin: 10200.0
max_pos                 : 204


In [2]:
from ib_insync import IB, Stock, MarketOrder, Order, Trade, BracketOrder
from mock_ib import MockIB
from dataclasses import dataclass, field
from typing import List, Optional, Union, Tuple
import pandas as pd
from copy import deepcopy
import stops, entries, trade_log
from datetime import datetime
import trades as tr


# def get_qty_shares(entry_price: float, stop_price: float, risk_vlaue: float) -> int:
#     """Calculate the position size based on risk percentage. Risk is a percentage of the account size expressd as a monetry value"""
#     return int(risk_vlaue / (entry_price - stop_price))

# @dataclass
# class TradeManager:
#     account_id: str
#     margin_allowance: float
#     max_account_pct: float
#     paper_trading: bool = True
#     balance_with_margin: float = 0
#     balance_withiout_margin: float = 0
#     equity: float = 0
#     margin: float = 0
#     realized_pnl: float = 0
#     unrealized_pnl: float = 0
#     active_trade_qty: int = 0
#     rr_ratio: float = 0
#     win_rate: float = 0

#     def __post_init__(self):
#         self.trades: List[BaseTrade] = []

#     def set_available_funds(self):
#         funds_margin = self.balance * self.max_account_pct
#         total_active_trade_value = sum([t.trade_value for t in self.trades if t.is_active])
#         self.balance_with_margin = funds_margin - total_active_trade_value

#     def get_position_size(self, entry_price: float, stop_price: float, risk_pct: float) -> int:
#         """Calculate the position size based on risk percentage, account size, margin, and max account pct"""
#         self.set_available_funds()
#         risk_value = self.balance_with_margin * risk_pct
#         pos_size = get_qty_shares(entry_price, stop_price, risk_value)
#         return pos_size





In [1]:
#! must use SB2 kernal to run this 

from ib_insync import *
util.startLoop()

ib = IB()


# Connect
ib.connect('127.0.0.1', 7496, clientId=2)

<IB connected to 127.0.0.1:7496 clientId=2>

In [4]:
import trades as tr

tx1 = tr.TradeX(ib=ib, symbol='TSM', barsize='1min', fund_allocation=1000)

In [6]:
import stops, entries

entrystrategy = entries.EntryStrategy(name='Hot Stuff', conditions=[ entries.PriceAbovePreviousHigh(), entries.LowerHighsPreviousBars(num_bars=2), entries.PriceNotBelowMA(ma_column='MA_cl_50', atr_column='ATR_10', atr_multiplier=0.0)])


stopStrategy = stops.StopLoss(
    init=stops.StopPrevBar(),
    # init=stops.StopGapDiff(3),
    # trail1=stops.StopPriorPiv(5),
    cond1=stops.CondDuration(5),
    trail1=stops.StopGapDiff(3),
    trail2=stops.StopPriorPiv(5),
    # trail2=stops.StopMA(21),
    cond2=stops.CondRRatio(2)
)

tx1.start_trade(entrystrategy, stopStrategy)
tx1.submit_bracket_order(quantity=1, stop_price=99.5,  target_price=None, outsideRth=True)

In [3]:
completed_orders = ib.reqCompletedOrders(apiOnly=False)
open_orders = ib.openOrders()
# Combine orders into a list of dictionaries
orders_data = []
for order in open_orders + completed_orders:
    orders_data.append({
        'OrderId': order.orderId,
        'Status': order.orderStatus.status,
        'Symbol': order.contract.symbol,
        'Action': order.action,
        'Quantity': order.totalQuantity,
        'Filled': order.filled,
        'Remaining': order.remaining,
        'AvgFillPrice': order.avgFillPrice
    })

# Create a DataFrame
df = pd.DataFrame(orders_data)
df

AttributeError: 'Trade' object has no attribute 'orderId'

In [4]:

tx1.__dict__

{'ib': <IB connected to 127.0.0.1:7496 clientId=1>,
 'symbol': 'TSM',
 'barsize': '1min',
 'direction': 'LONG',
 'status': 'ENTRY_SUBMITTED',
 'is_active': False,
 'chart_time': datetime.datetime(2024, 11, 15, 7, 21, 7, 857515),
 'real_time': datetime.datetime(2024, 11, 15, 7, 21, 7, 857515),
 'entry_time': None,
 'exit_time': None,
 'duration': None,
 'entry_name': None,
 'entry_av_price': None,
 'entry_filled': (0, 100),
 'entry_ib_id': '7',
 'entry_ib_status': 'PendingSubmit',
 'exit_name': None,
 'exit_av_price': None,
 'exit_filled': (0, 100),
 'exit_type': None,
 'exit_ib_id': None,
 'exit_ib_status': None,
 'position': 0,
 'value': 0.0,
 'close_price': None,
 'stop_name': None,
 'stop_price': None,
 'stop_filled': (0, 0),
 'stop_ib_id': '8',
 'stop_ib_status': 'PendingSubmit',
 'target_name': None,
 'target_price': None,
 'target_filled': (0, 0),
 'target_ib_id': None,
 'target_ib_status': None,
 'unrealized_pl': 0.0,
 'realized_pl': 0.0,
 'target_risk_reward': 0.0,
 'actual_ris

In [ ]:
tx1.entry_order.__dict__

{'contract': Stock(symbol='TSM', exchange='SMART', currency='USD'),
 'order': Order(orderId=3, clientId=1, permId=679026236, action='BUY', totalQuantity=100.0, orderType='MKT', lmtPrice=0.0, auxPrice=0.0, transmit=False),
 'orderStatus': OrderStatus(orderId=3, status='PreSubmitted', filled=0.0, remaining=100.0, avgFillPrice=0.0, permId=679026236, parentId=0, lastFillPrice=0.0, clientId=1, whyHeld='', mktCapPrice=0.0),
 'fills': [],
 'log': [TradeLogEntry(time=datetime.datetime(2024, 11, 15, 7, 5, 4, 367177, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0),
  TradeLogEntry(time=datetime.datetime(2024, 11, 15, 7, 5, 4, 760364, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0)],
 'advancedError': '',
 'statusEvent': Event<statusEvent, []>,
 'modifyEvent': Event<modifyEvent, []>,
 'fillEvent': Event<fillEvent, []>,
 'commissionReportEvent': Event<commissionReportEvent, []>,
 'filledEvent': Event<filledEvent, []>,
 'cancelEvent': E

In [5]:
def monitor_order_fills(self) -> None:
        """
        Extracts fill information from entry, stop and target orders and maps them
        to the appropriate class attributes.
        """
        if self.entry_order:
            print(f"Entry Order Status: {self.entry_order.orderStatus.status}")
            self.entry_ib_status = self.entry_order.orderStatus.status
            # Check entry order fills
            fills = self.entry_order.fills
            if fills:
                total_shares_filled = sum(fill.execution.shares for fill in fills)
                self.entry_filled = (total_shares_filled, self.entry_order.totalQuantity)
                self.av_entry_price = sum(fill.execution.shares * fill.execution.price for fill in fills) / total_shares_filled if total_shares_filled > 0 else None
                self.entry_time = fills[-1].execution.time if fills else None
                print(f"Entry Order Status: Filled {total_shares_filled} of {self.entry_order.totalQuantity} shares")
                
        if self.stop_order:
            self.stop_ib_status = self.stop_order.orderStatus.status
            # Check stop order fills
            fills = self.stop_order.fills
            if fills:
                total_shares_filled = sum(fill.execution.shares for fill in fills)
                self.exit_filled = (total_shares_filled, self.stop_order.totalQuantity)
                self.av_exit_price = sum(fill.execution.shares * fill.execution.price for fill in fills) / total_shares_filled if total_shares_filled > 0 else None
                self.exit_time = fills[-1].execution.time if fills else None
                self.exit_type = "STOP"
                print(f"Stop Order Status: Filled {total_shares_filled} of {self.stop_order.totalQuantity} shares")

        if self.target_order:
            self.target_ib_status = self.target_order.orderStatus.status
            # Check target order fills
            fills = self.target_order.fills
            if fills:
                total_shares_filled = sum(fill.execution.shares for fill in fills)
                self.exit_filled = (total_shares_filled, self.target_order.totalQuantity)
                self.av_exit_price = sum(fill.execution.shares * fill.execution.price for fill in fills) / total_shares_filled if total_shares_filled > 0 else None
                self.exit_time = fills[-1].execution.time if fills else None
                self.exit_type = "TARGET"
                print(f"Target Order Status: Filled {total_shares_filled} of {self.target_order.totalQuantity} shares")

monitor_order_fills(tx1)

Entry Order Status: Inactive


In [6]:
tx1.__dict__

{'ib': <IB connected to 127.0.0.1:7496 clientId=1>,
 'symbol': 'TSM',
 'barsize': '1min',
 'direction': 'LONG',
 'status': 'ENTRY_SUBMITTED',
 'is_active': False,
 'chart_time': datetime.datetime(2024, 11, 15, 7, 21, 7, 857515),
 'real_time': datetime.datetime(2024, 11, 15, 7, 21, 7, 857515),
 'entry_time': None,
 'exit_time': None,
 'duration': None,
 'entry_name': None,
 'entry_av_price': None,
 'entry_filled': (0, 100),
 'entry_ib_id': '7',
 'entry_ib_status': 'Inactive',
 'exit_name': None,
 'exit_av_price': None,
 'exit_filled': (0, 100),
 'exit_type': None,
 'exit_ib_id': None,
 'exit_ib_status': None,
 'position': 0,
 'value': 0.0,
 'close_price': None,
 'stop_name': None,
 'stop_price': None,
 'stop_filled': (0, 0),
 'stop_ib_id': '8',
 'stop_ib_status': 'Cancelled',
 'target_name': None,
 'target_price': None,
 'target_filled': (0, 0),
 'target_ib_id': None,
 'target_ib_status': None,
 'unrealized_pl': 0.0,
 'realized_pl': 0.0,
 'target_risk_reward': 0.0,
 'actual_risk_reward'

In [8]:
from ib_insync import IB, Stock
from datetime import datetime
import pytz

def is_open_for_trading(ib, symbol: str) -> bool:
    """
    Check if current time is outside today's trading hours for a given symbol.
    Prints current time and trading hours in exchange timezone.
    """
    details = ib.reqContractDetails(Stock(symbol, 'SMART', 'USD'))[0]
    exchange_tz = pytz.timezone(details.timeZoneId)
    current_time = datetime.now(exchange_tz)
    today = current_time.strftime('%Y%m%d')
    
    # Find today's trading period
    today_period = next(
        (period for period in details.tradingHours.split(';')
         if period.startswith(today) and 'CLOSED' not in period),
        None
    )
    
    if not today_period:
        return False  # No trading today
        
    start_str, end_str = today_period.split('-')
    start_time = exchange_tz.localize(datetime.strptime(start_str, '%Y%m%d:%H%M'))
    end_time = exchange_tz.localize(datetime.strptime(end_str, '%Y%m%d:%H%M'))
    
    print(f"Exchange time now: {current_time.strftime('%H:%M:%S')}")
    print(f"Market opens at:   {start_time.strftime('%H:%M:%S')}")
    print(f"Market closes at:  {end_time.strftime('%H:%M:%S')}")
    
    return (start_time <= current_time <= end_time)

# Example usage
symbol = 'TSM'
local_timezone = 'America/New_York'
is_open_for_trading(ib, symbol)

Exchange time now: 16:58:21
Market opens at:   04:00:00
Market closes at:  20:00:00


True

In [4]:
ib

Task exception was never retrieved
future: <Task finished name='Task-5' coro=<IB.connectAsync() done, defined at c:\Users\sidsu\anaconda3\envs\SB4\Lib\site-packages\ib_insync\ib.py:1739> exception=ConnectionRefusedError(10061, "Connect call failed ('127.0.0.1', 7497)")>
Traceback (most recent call last):
  File "c:\Users\sidsu\anaconda3\envs\SB4\Lib\site-packages\ib_insync\ib.py", line 1748, in connectAsync
    await self.client.connectAsync(host, port, clientId, timeout)
  File "c:\Users\sidsu\anaconda3\envs\SB4\Lib\site-packages\ib_insync\client.py", line 211, in connectAsync
    await asyncio.wait_for(self.conn.connectAsync(host, port), timeout)
  File "c:\Users\sidsu\anaconda3\envs\SB4\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\sidsu\anaconda3\envs\SB4\Lib\site-packages\ib_insync\connection.py", line 39, in connectAsync
    self.transport, _ = await loop.create_connection(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^

NameError: name 'ib' is not defined